In [1]:
import json
import pandas as pd

In [2]:
tags_query = """
SELECT
  table_schema,
  table_name,
  REGEXP_EXTRACT(option_value, r'"schedule",\s*"([^"]+)"') as schedule
FROM
 `passculture-data-prod`.`region-europe-west1`.INFORMATION_SCHEMA.TABLE_OPTIONS
WHERE option_name = 'labels';
"""

In [4]:
table_schedule_df = pd.read_gbq(tags_query)

In [7]:
dataset = 'analytics_prod'

In [8]:
last_update_query = f"""
SELECT 
    table_id as table_name,
    dataset_id as table_schema,
    TIMESTAMP_MILLIS(last_modified_time) as last_modified_time
FROM {dataset}.__TABLES__
"""


In [9]:
table_last_update_df = pd.read_gbq(last_update_query)


In [1]:
import pandas as pd
def get_datasets_to_scan():
    query = """SELECT
        distinct schema_name
        FROM `passculture-data-prod`.`region-europe-west1`.INFORMATION_SCHEMA.SCHEMATA;
    """
    df = pd.read_gbq(query)

    return df['schema_name'].tolist()
    datasets_to_scan = [dataset for dataset in datasets_to_scan if dataset.startswith('int_') or dataset in ["analytics_prod", 'clean_prod', 'raw_prod', 'backend_prod']]
datasets_to_scan = get_datasets_to_scan()

/home/luciler/.pyenv/versions/3.10.4/lib/python3.10/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [55]:
bigquery_df  = pd.concat([
    get_last_modified_date_table(dataset) for dataset in datasets_to_scan
    ], axis=0).reset_index(drop=True)

In [64]:
df = bigquery_df.merge(relations, on=["dataset_id", "table_id"], how='left')

In [72]:
import datetime 
today = datetime.datetime.now()
days3 = datetime.timedelta(days = 3)
days7 = datetime.timedelta(days = 7)
month = datetime.timedelta(weeks = 4)

In [73]:
mapping = {
    "daily": today - days3,
    "weekly": today - days7,
    "monthly": today - month,
    "default": today - month
}

In [74]:
mapping

{'daily': datetime.datetime(2024, 8, 10, 17, 58, 13, 48070),
 'weekly': datetime.datetime(2024, 8, 6, 17, 58, 13, 48070),
 'monthly': datetime.datetime(2024, 7, 16, 17, 58, 13, 48070),
 'default': datetime.datetime(2024, 7, 16, 17, 58, 13, 48070)}

In [78]:
df['tag'].fillna('default', inplace=True)

In [83]:
df['last_modified_time'] = pd.to_datetime(df['last_modified_time']).dt.tz_localize(None)

In [84]:
warning_tables = df[df['last_modified_time'] < df['tag'].map(mapping)]

In [87]:
import re

In [95]:
def contains_valid_date(string):
    pattern = r'(\d{4})(\d{2})(\d{2})'
    
    matches = re.findall(pattern, string)
    
    for match in matches:
        year, month, day = match
        try:
            datetime.datetime(int(year), int(month), int(day))
            return True
        except ValueError:
            continue
    
    return False

In [96]:
warning_tables['is_partition_table'] = warning_tables['table_id'].apply(contains_valid_date)

/tmp/ipykernel_16702/1761924537.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  warning_tables['is_partition_table'] = warning_tables['table_id'].apply(contains_valid_date)


In [98]:
warning_tables[warning_tables.is_partition_table == False]

,table_id,dataset_id,last_modified_time,relation_name,alias,tag,dataset_name,is_dbt_managed,is_partition_table
113,adage_involved_student_tmp,clean_prod,2024-06-24 09:17:41.892,NaN,NaN,default,NaN,NaN,False
114,api_logs,clean_prod,2024-03-27 20:26:22.850,NaN,NaN,default,NaN,NaN,False
239,offer_item_ids_copy,clean_prod,2024-07-12 10:40:27.221,NaN,NaN,default,NaN,NaN,False
248,rp2019_indcvi,clean_prod,2022-12-13 19:50:28.035,NaN,NaN,default,NaN,NaN,False
2183,attribution,int_firebase_prod,2024-04-04 12:25:14.453,NaN,NaN,default,NaN,NaN,False
2210,aggregated_weekly_cultural_partner_activity,analytics_prod,2024-03-18 10:48:59.264,NaN,NaN,default,NaN,NaN,False
2212,api_logs,analytics_prod,2024-03-28 01:01:16.782,NaN,NaN,default,NaN,NaN,False
2237,backend_events,analytics_prod,2024-03-27 20:29:55.630,NaN,NaN,default,NaN,NaN,False
2238,backend_logs,analytics_prod,2024-04-15 15:28:00.008,NaN,NaN,default,NaN,NaN,False
2242,bookable_offer_history,analytics_prod,2024-04-24 06:52:57.764,NaN,NaN,default,NaN,NaN,False


In [19]:
models

- relation_name : gcp_project_id.dataset_id.table_id
- alias
- tags

SyntaxError: illegal target for annotation (751834372.py, line 3)

In [ ]:
manifest.json

> nodes
>> resource_type = 'model'

get 
- relation_name : gcp_project_id.dataset_id.table_id
- alias
- tags



